In [12]:
import pandas as pd
import numpy as np
import os

In [6]:
df=pd.read_parquet(f'D:\\2min-resample\MetaDataSeparation\\051KyAT8XBCFjoIDTJGQ+Q==.asfreqffill.2min.snappy.parquet')
print(df.head())

In [8]:



def get_unique_columns(folder_path, output_csv_path):
    # 初始化一个空的集合用于存储唯一的列名
    unique_columns = set()

    # 获取文件夹中所有文件的列表
    files = os.listdir(folder_path)

    # 过滤列表，只包含 parquet 文件
    parquet_files = [f for f in files if f.endswith('.parquet')]

    for file in parquet_files:
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_parquet(file_path)
            # 更新唯一列名集合
            unique_columns.update(df.columns)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    # 将唯一列名转换为数据框
    unique_columns_df = pd.DataFrame(list(unique_columns), columns=['Column Name'])

    # 保存到 CSV 文件
    unique_columns_df.to_csv(output_csv_path, index=False)
    print(f"Unique column names saved to {output_csv_path}")

# 示例使用
get_unique_columns('D:\\2min-resample', 'D:\\2min-resample\MetaDataSeparation\\unique_columns.csv')

In [19]:
def convert_parquet_to_csv(file_path):
    df=pd.read_parquet(file_path)
    csv_file_path = os.path.splitext(file_path)[0] + '.csv'
    df.to_csv(csv_file_path, index=True)
    
convert_parquet_to_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\With_RetT\processed_files_Version3\+oMphykhj0iuwq6Ah8RVIQ==.asfreqffill.2min.snappy.parquet')

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
def is_data_intact(daily_data):
    expected_count = 24 * 30  # 24 hours * 30 (2-minute intervals per hour)
    return len(daily_data) == expected_count

# Function to plot daily consumption curve
def plot_daily_consumption(file_path):
    # Load the data from parquet file
    data = pd.read_parquet(file_path)
    
    # Ensure the index is in datetime format
    data.index = pd.to_datetime(data.index)
    
    # Find days with complete data
    days = data.resample('D').apply(is_data_intact)
    intact_days = days[days].index
    
    if intact_days.empty:
        print("No days with complete data found.")
        return
    
    # Select a random date with complete data
    random_date = np.random.choice(intact_days)
    
    # Extract the data for the selected date
    daily_curve = data.loc[random_date.strftime('%Y-%m-%d')]
    
    # Plot the daily consumption curve
    plt.figure(figsize=(10, 6))
    plt.plot(daily_curve.index, daily_curve['ActPow[%](float32)'], marker='o', linestyle='-')
    plt.title(f'Daily Gas Consumption Curve for {random_date.strftime("%Y-%m-%d")}')
    plt.xlabel('Time')
    plt.ylabel('Gas Consumption')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Example usage
file_path = 'D:\\2min-resample\MetaDataSeparation\MetaData Filtered\With_RetT\processed_files_Version2\\+8KRfSBu4OUWy3Nmn6xQDg==.asfreqffill.2min.snappy.parquet'  # Replace with the actual file path
plot_daily_consumption(file_path)

In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Function to check if the data is intact for a given day
def is_data_intact(daily_data, frequency='30min'):
    # Expected count of intervals for a full day with the given frequency
    if frequency == '30min':
        expected_count = 24 * 2  # 24 hours * 2 (half-hourly intervals per hour)
    else:
        raise ValueError("Unsupported frequency")
    
    # Check if there are any missing 2-minute data points within the day
    intact = (len(daily_data) == expected_count and not daily_data.isnull().values.any())
    return intact

# Function to plot daily consumption curve
def plot_random_days(file_path, num_days=12):
    # Load the data from parquet file
    data = pd.read_parquet(file_path)
    
    # Ensure the index is in datetime format
    data.index = pd.to_datetime(data.index)
    data = data[['ActPow[%](float32)']]
    # print(data.head())
    # Resample the data to half-hourly frequency and calculate the mean consumption
    half_hourly_data = data.resample('30min').mean()
    # print(half_hourly_data.head())
    # half_hourly_data.to_csv("D:\\2min-resample\MetaDataSeparation\MetaData Filtered\With_RetT\processed_files_Version2\\test.csv", index=True)
    # Normalize to get unique days and check data integrity for each day
    intact_days = []
    print(f'there are {len(half_hourly_data.index.normalize().unique())} days in the profile')
    for day in half_hourly_data.index.normalize().unique():
        daily_data = half_hourly_data.loc[day.strftime('%Y-%m-%d')]
        if is_data_intact(daily_data):
            intact_days.append(day)
    
    if len(intact_days) < num_days:
        print(f"Only {len(intact_days)} intact days available. Plotting all available intact days.")
        num_days = len(intact_days)
    print(f' There are  {len(intact_days)} intact days in the profile')
    # Select random intact days
    random_days = np.random.choice(intact_days, num_days, replace=False)
    
    # Plot the consumption curve for each selected day
    for random_date in random_days:
        daily_curve = half_hourly_data.loc[random_date.strftime('%Y-%m-%d')]
        # print(daily_curve)
        plt.figure(figsize=(10, 6))
        plt.plot(daily_curve.index, daily_curve['ActPow[%](float32)'], marker='o', linestyle='-')
        plt.title(f'Half-Hourly Gas Consumption Curve for {random_date.strftime("%Y-%m-%d")}')
        plt.xlabel('Time')
        plt.ylabel('Average Gas Consumption')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# Example usage
file_path = r"D:\2min-resample\MetaDataSeparation\MetaData Filtered\With_RetT\processed_files_Version3\+flXTFV6kC6oYsBh3pYvgA==.asfreqffill.2min.snappy.parquet"  # Replace with the actual file path
plot_random_days(file_path, num_days=12)

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Function to check if the data is intact for a given day
def is_data_intact(daily_data, interval='2T'):
    # Expected count of intervals for a full day with the given interval
    expected_count = 24 * (60 // int(interval[:-1]))  # 24 hours * number of intervals per hour
    # Check if there are any missing data points within the day
    return len(daily_data) == expected_count and not daily_data.isnull().values.any()

# Function to plot daily consumption curve
def plot_january_days(file_path):
    # Load the data from parquet file
    data = pd.read_parquet(file_path)
    
    # Ensure the index is in datetime format
    data.index = pd.to_datetime(data.index)
    data = data[['ActPow[%](float32)']]
    
    # Filter the data to only include January
    january_data = data[data.index.month == 1]
    
    # Resample the data to half-hourly frequency and calculate the mean consumption
    half_hourly_data = january_data.resample('30T').mean()
    
    # Normalize to get unique days and check data integrity for each day
    intact_days = []
    for day in half_hourly_data.index.normalize().unique():
        daily_data = data.loc[day.strftime('%Y-%m-%d')]
        if is_data_intact(daily_data):
            intact_days.append(day)
    
    if not intact_days:
        print("No intact days available in January.")
        return
    
    # Plot the consumption curve for each intact day in January
    for random_date in intact_days:
        daily_curve = half_hourly_data.loc[random_date.strftime('%Y-%m-%d')]
        
        plt.figure(figsize=(10, 6))
        plt.plot(daily_curve.index, daily_curve['ActPow[%](float32)'], marker='o', linestyle='-')
        plt.title(f'Half-Hourly Gas Consumption Curve for {random_date.strftime("%Y-%m-%d")}')
        plt.xlabel('Time')
        plt.ylabel('Average Gas Consumption')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# Example usage
file_path = 'D:\\2min-resample\\MetaDataSeparation\\MetaData Filtered\\With_RetT\\processed_files_Version3\\_PkmtPEvUTf4V2E54UrlCA==.asfreqffill.2min.snappy.parquet'  # Replace with the actual file path
plot_january_days(file_path)

In [54]:
import os
import pandas as pd
import glob
import pyarrow.parquet as pq
from tqdm import tqdm
import sys

def extract_and_resample(file, column_name='ActPow[%](float32)', resample_freq='30min'):
    df = pq.read_table(file).to_pandas()
    df.index = pd.to_datetime(df.index)
    resampled_df = df[column_name].resample(resample_freq).mean()
    return resampled_df

def create_common_time_index(file_list, resample_freq='30min'):
    # List to hold all the indices
    all_indices = []

    # Collect indices from all files
    for file in file_list:
        df = pq.read_table(file).to_pandas()
        df.index = pd.to_datetime(df.index)
        resampled_index = df.resample(resample_freq).sum().index
        all_indices.append(pd.DataFrame(index=resampled_index))

    # Concatenate all indices and generate the common time index
    all_indices_df = pd.concat(all_indices)
    common_time_index = all_indices_df.index.drop_duplicates().sort_values()

    return common_time_index

def main():
    # Directory containing the parquet files
    directory = r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\WO_RetT\processed_files_Version3\\'
    file_list = glob.glob(os.path.join(directory, '*.parquet'))

    # Create a common time index
    common_time_index = create_common_time_index(file_list)

    # Dictionary to hold all resampled data
    data_dict = {}

    # Process each file
    for file in tqdm(file_list, desc="Processing files", file=sys.stdout):
        file_name = os.path.basename(file).replace('.parquet', '')
        resampled_series = extract_and_resample(file)
        
        # Align the resampled series to the common time index without filling missing values
        resampled_series = resampled_series.reindex(common_time_index)
        data_dict[file_name] = resampled_series

    # Combine all series into a single DataFrame
    combined_df = pd.DataFrame(data_dict, index=common_time_index)

    # Save the combined DataFrame to a CSV file
    output_csv_path = os.path.join(directory, 'combined_consumption_WO.csv')
    combined_df.to_csv(output_csv_path, index=True)

if __name__ == "__main__":
    main()

In [57]:
df1 = pd.read_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\WO_RetT\processed_files_Version3\\combined_consumption_WO.csv',index_col=0)
df2 = pd.read_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\With_RetT\processed_files_Version3\\combined_consumption_With.csv',index_col=0)

df3 = df1.join(df2,how='outer')
df3.to_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\combined_consumption.csv')

In [61]:
df4 = pd.read_csv(r"D:\\2min-resample\MetaDataSeparation\MetaData Filtered\PowerMax.csv",index_col=0)
df4 = df4.T
df4.to_csv(r"D:\\2min-resample\MetaDataSeparation\MetaData Filtered\PowerMaxT.csv")

In [62]:
df4 = pd.read_csv(r"D:\\2min-resample\MetaDataSeparation\MetaData Filtered\PowerMaxT.csv",index_col=0)
df4 = df4.dropna(axis=1, how='all')
df4.to_csv(r"D:\\2min-resample\MetaDataSeparation\MetaData Filtered\PowerMaxT.csv")


In [74]:
df_multipliers = pd.read_csv(r"D:\\2min-resample\MetaDataSeparation\MetaData Filtered\PowerMaxT.csv",index_col=0)
df_data = pd.read_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\combined_consumption_per1.csv',index_col=0)

# 确保 df_multipliers 只有一行
if len(df_multipliers) != 1:
    raise ValueError("df_multipliers should have only one row")

# 获取乘数的唯一行
multiplier_row = df_multipliers.iloc[0]

# 对齐两个 DataFrame 的列名
common_columns = df_data.columns.intersection(multiplier_row.index)
df_data_aligned = df_data[common_columns]
multiplier_row_aligned = multiplier_row[common_columns]

# 每一列乘以对应列名中的唯一行的元素
df_result = df_data_aligned.multiply(multiplier_row_aligned, axis=1)

# 将所有元素除以200
df_result = df_result / 200

df_result.to_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\combined_consumption_kWh.csv')

In [10]:
import pandas as pd

# Assuming your data is in a DataFrame called 'df' with a datetime index and a column 'consumption' for electricity usage
df = pd.read_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\load duration model_WODHW\kW1.75\sum.csv',index_col=0)
df.index = pd.to_datetime(df.index)
# Filter data for the months November to March
df_filtered = df[(df.index.month >= 11) | (df.index.month <= 3)]

# Function to get the peak in each half of the day and the daily minimum
def get_peaks_and_min(group):
    # Divide the day into two halves
    morning = pd.DataFrame(group.between_time('00:00', '11:59'))
    evening = pd.DataFrame(group.between_time('12:00', '23:59'))
    print(morning)
    # Find the peak for the morning half
    peak_morning = morning.loc[morning['Sum'].idxmax()]
    
    # Find the peak for the evening half
    peak_evening = evening.loc[evening['Sum'].idxmax()]
    
    # Find the overall minimum value
    min_val = group.loc[group['Sum'].idxmin()]
    
    return pd.Series({
        'peak_morning_value': peak_morning['Sum'],
        'peak_morning_time': peak_morning.name,
        'peak_evening_value': peak_evening['Sum'],
        'peak_evening_time': peak_evening.name,
        'min_value': min_val['Sum'],
        'min_time': min_val.name
    })

# Resample to daily frequency and apply the function
result = df_filtered['Sum'].resample('D').apply(get_peaks_and_min)

# Reset the index for better readability
result = result.reset_index()

result.to_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\load duration model_WODHW\kW1.75\sum11.csv')

# Display the results
print(result)

                                   Sum
2021-11-01 00:00:00+00:00   227.375467
2021-11-01 00:30:00+00:00   216.057333
2021-11-01 01:00:00+00:00   212.673867
2021-11-01 01:30:00+00:00   209.092000
2021-11-01 02:00:00+00:00   214.171733
2021-11-01 02:30:00+00:00   229.376267
2021-11-01 03:00:00+00:00   231.607200
2021-11-01 03:30:00+00:00   253.145255
2021-11-01 04:00:00+00:00   298.634300
2021-11-01 04:30:00+00:00   370.825600
2021-11-01 05:00:00+00:00   549.034817
2021-11-01 05:30:00+00:00   702.034133
2021-11-01 06:00:00+00:00   995.032890
2021-11-01 06:30:00+00:00  1277.905277
2021-11-01 07:00:00+00:00  1408.027076
2021-11-01 07:30:00+00:00  1393.634764
2021-11-01 08:00:00+00:00  1252.669550
2021-11-01 08:30:00+00:00  1042.189974
2021-11-01 09:00:00+00:00   853.643052
2021-11-01 09:30:00+00:00   761.907837
2021-11-01 10:00:00+00:00   712.121226
2021-11-01 10:30:00+00:00   658.670915
2021-11-01 11:00:00+00:00   665.225253
2021-11-01 11:30:00+00:00   674.723917
                         

KeyError: 'Sum'

In [12]:
def get_peaks_and_min(group):
    # Check and possibly rename the column for consistency
    if 'Sum' not in group.columns:
        # Assuming the column has been renamed or altered
        correct_column = group.columns[0]  # or however you identify the correct column
        group = group.rename(columns={correct_column: 'Sum'})
    
    # Morning time slice (midnight to noon)
    morning = group.between_time('00:00', '11:59')
    print("Morning Data Slice:\n", morning.head(10))

    # Evening time slice (noon to midnight)
    evening = group.between_time('12:00', '23:59')
    print("Evening Data Slice:\n", evening.head(10))

    # Find the peak for the morning half
    peak_morning = morning.loc[morning['Sum'].idxmax()]

    # Find the peak for the evening half
    peak_evening = evening.loc[evening['Sum'].idxmax()]

    # Find the overall minimum value
    min_val = group.loc[group['Sum'].idxmin()]

    return pd.Series({
        'peak_morning_value': peak_morning['Sum'],
        'peak_morning_time': peak_morning.name,
        'peak_evening_value': peak_evening['Sum'],
        'peak_evening_time': peak_evening.name,
        'min_value': min_val['Sum'],
        'min_time': min_val.name
    })

# Applying to the filtered data
result = df_filtered.resample('D').apply(get_peaks_and_min)
result.to_csv(r'D:\2min-resample\MetaDataSeparation\MetaData Filtered\load duration model_WODHW\kW1.75\sum11.csv')
print(result)

Morning Data Slice:
                                   Sum
2021-11-01 00:00:00+00:00  227.375467
2021-11-01 00:30:00+00:00  216.057333
2021-11-01 01:00:00+00:00  212.673867
2021-11-01 01:30:00+00:00  209.092000
2021-11-01 02:00:00+00:00  214.171733
2021-11-01 02:30:00+00:00  229.376267
2021-11-01 03:00:00+00:00  231.607200
2021-11-01 03:30:00+00:00  253.145255
2021-11-01 04:00:00+00:00  298.634300
2021-11-01 04:30:00+00:00  370.825600
Evening Data Slice:
                                    Sum
2021-11-01 12:00:00+00:00   730.541007
2021-11-01 12:30:00+00:00   714.428195
2021-11-01 13:00:00+00:00   710.932611
2021-11-01 13:30:00+00:00   737.866156
2021-11-01 14:00:00+00:00   748.108745
2021-11-01 14:30:00+00:00   812.979831
2021-11-01 15:00:00+00:00   882.282815
2021-11-01 15:30:00+00:00  1056.481692
2021-11-01 16:00:00+00:00  1313.550892
2021-11-01 16:30:00+00:00  1522.106823
Morning Data Slice:
                                   Sum
2021-11-02 00:00:00+00:00  250.395448
2021-11-02 00: